<a href="https://colab.research.google.com/github/Aira-alice/Ai-Voice/blob/main/AI%20VOICE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code aims to make Gemini AI able to use voice input very well and produce very good voice output.

## ***run in order***

In [ ]:
# @title 1
!pip install tts
!pip install google-generativeai
!pip install ffmpeg-python
!pip install -U openai-whisper

#untuk menyelesaikan error yang disebabkan oleh cutlet
import locale
from IPython.display import clear_output
!pip install cutlet
!pip install unidic-lite

clear_output()

In [ ]:
# @title 2
import google.generativeai as genai
from IPython.display import clear_output


import scipy
import whisper
import time

import IPython
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

In [ ]:
# @title 3
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

# List available 🐸TTS models
print(TTS().list_models())

# Init TTS
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)


clear_output()

In [ ]:
# @title 4
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
# @title 5
import google.generativeai as genai

from IPython.display import clear_output
genai.configure(api_key="your Api Key")

prompt = "role_system:name: Alisa\nrole: Girlfriend\npersonality: Tsundere girl"


# Set up the model
generation_config = {
  "temperature": 0.7,
  "top_p": 1,
  "top_k": 1,
  "max_output_tokens": 2048,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_NONE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_NONE"
  },
]


modelAi = genai.GenerativeModel(model_name='gemini-1.5-flash-latest',
                              generation_config=generation_config,
                              safety_settings=safety_settings,
                              system_instruction=prompt)

In [ ]:
# please provide a voice sample to imitate

cloneVoice = "2B NEW.mp3"

In [ ]:
model = whisper.load_model("medium")
clear_output()

In [ ]:
# @title play with speech recognition
while True :
  audio, sr = get_audio()
  scipy.io.wavfile.write('record.wav', sr, audio)

  result = model.transcribe("record.wav",language="id")

  prompt_parts = (result["text"])
  print("User : " + prompt_parts)
  response = modelAi.generate_content(prompt_parts)

  print("AI : " + response.text )



  tts.tts_to_file(text= response.text , speaker_wav=cloneVoice, language="ja", file_path="output.wav")

  time.sleep(1)
  wn = Audio('output.wav', autoplay=True)
  display(wn)
  inp = input("")
  if inp == "break" :
    clear_output()
    break
  clear_output()

In [ ]:
# @title play input teks
while True :
  #audio, sr = get_audio()
  #scipy.io.wavfile.write('record.wav', sr, audio)
  #result = model.transcribe("record.wav")

  #prompt_parts = (result["text"])
  prompt_parts = input("User : ")
  print("User : " + prompt_parts)
  response = modelAi.generate_content(prompt_parts)


  print("AI : " + response.text )



  tts.tts_to_file(text= response.text , speaker_wav=cloneVoice, language="ja", file_path="output.wav")

  time.sleep(1)
  wn = Audio('output.wav', autoplay=True)
  display(wn)
  inp = input("")
  if inp == "break" :
    clear_output()
    break
  clear_output()